## This notebook walks through processing WQP data using the harmonize_wq package. It loads a larger set of Gulf of Mexico Esturaries, running on one at a time. It does not dig into report and nauances of results as much.


##### import the required libraries. Check requirements.txt for dependencies that should be installed.

In [ ]:
import os
import pandas
import geopandas
from harmonize_wq import harmonize
from harmonize_wq import convert
from harmonize_wq import wrangle
from harmonize_wq import clean
import dataretrieval.wqp as wqp

#### Download location data using dataretrieval

In [ ]:
# Read geometry for Area of Interest from local file
i = 1  #Index for the estuary to retrieve (UPDATE EACH TIME)

# If saving results locally set file names for outputs
out_dir = r'D:\Local_GIS\NCCA'  # UPDATE ONCE WITH TEMP DIRECTORY
aoi_dir = os.path.join(out_dir, r'NCCA_2020_dissolvedon_EDACDA_NM.shp')
aoi_gdf_all = geopandas.read_file(aoi_dir)

# WGS 1984 for WQP query
aoi_gdf_all = aoi_gdf_all.to_crs(epsg=4326)

In [ ]:
# Plot all the Gulf of Mexico Estuaries
aoi_gdf_all.plot()

In [ ]:
# Print reformated estuary name
estuary_name = aoi_gdf_all.iloc[i]['EDACDA_NM']
out_est_name = 'Estuary_' + str(estuary_name).replace(" ", "_")
out_est_name = out_est_name.replace(".", "")
out_est_name = out_est_name.replace("(", "")
out_est_name = out_est_name.replace(")", "")
print('Estuary Name: "{}" -> "{}"'.format(estuary_name, out_est_name))

In [ ]:
# Get polygon from polygons
aoi_gdf = aoi_gdf_all.loc[[i],'geometry']
#aoi_gdf = aoi_gdf_all.iloc[i]['geometry']
#aoi_gdf['geometry'] = geopandas.GeoDataFrame(aoi_gdf, crs="EPSG:4326")

In [ ]:
# Geometry for selection
aoi_gdf.plot()

In [ ]:
# Spatial query parameters
# Each estuary may be multi-polygon, so the query will be built around the full extent
#instead of one since polygon
bBox = ','.join(map(str, aoi_gdf.total_bounds))  #get bBox string for total extent of all

In [ ]:
# Build query
query = {'characteristicName': ['Phosphorus',
                                'Temperature, water',
                                'Depth, Secchi disk depth',
                                'Dissolved oxygen (DO)',
                                'Salinity',
                                'pH',
                                'Nitrogen',
                                'Conductivity',
                                'Organic carbon',
                                'Chlorophyll a',
                                'Turbidity',
                                'Sediment',
                                'Fecal Coliform',
                                'Escherichia coli']}
query['bBox'] = bBox

In [ ]:
# Query stations (can be slow)
stations, site_md = wqp.what_sites(**query)

In [ ]:
# Harmonize location datums to 4326
stations_gdf = harmonize.harmonize_locations(stations, outEPSG=4326)

In [ ]:
# Clip it to area of interest
stations_clipped = geopandas.clip(stations_gdf, aoi_gdf)

In [ ]:
# Save it
out_geo = os.path.join(out_dir, out_est_name + ".shp")
#stations_clipped.to_file(out_geo)

In [ ]:
# Map it
stations_clipped.plot()

#### Harmonize characteristic data (all at once)

In [ ]:
# Query results
query['dataProfile'] = 'narrowResult'
res_narrow, md_narrow = wqp.get_results(**query)

In [ ]:
df = res_narrow
# Save it
out_df = os.path.join(out_dir, out_est_name + ".csv")
df.to_csv(out_df,index=False)
# Look at it
df

The harmonize_all() function identifies the characteristics present and uses preset defaults to harmonize each. This function does not has as much flexibility e.g., to keep intermediate columns, produce reports, or convert to non-default units.

In [ ]:
# Harmonize all
# Note that errors='skip' or 'ignore' will be needed to supress errors in dimensionality in some cases
# such errors occur when a unit can not be converted to the desired unit (e.g., degC to m)
df = harmonize.harmonize_all(df)
df

In [ ]:
# Set standard columns to look through results
cols = ['ResultMeasureValue', 'ResultMeasure/MeasureUnitCode', 'QA_flag']

Note: if there were no results for a given characteristic a result column will not be generated for that characteristic and there will be a keyError when trying to look at results, e.g., 'KeyError: "['Conductivity'] not in index"' if there are no conductivity results

In [ ]:
# Secchi
df.loc[df['CharacteristicName']=='Depth, Secchi disk depth', cols + ['Secchi']]

In [ ]:
# Temperature
df.loc[df['CharacteristicName']=='Temperature, water', cols + ['Temperature']]

In [ ]:
# Dissolved Oxygen
df.loc[df['CharacteristicName']=='Dissolved oxygen (DO)', cols + ['DO']]

In [ ]:
# pH
df.loc[df['CharacteristicName']=='pH', cols + ['pH']]

In [ ]:
# Salinity
df.loc[df['CharacteristicName']=='Salinity', cols + ['Salinity']]

In [ ]:
# Nitrogen
df.loc[df['CharacteristicName']=='Nitrogen', cols + ['Nitrogen']]

In [ ]:
# Conductivity
df.loc[df['CharacteristicName']=='Conductivity', cols + ['Conductivity']]

In [ ]:
# Chlorophyll A
df.loc[df['CharacteristicName']=='Chlorophyll a', cols + ['Chlorophyll']]

In [ ]:
# Carbon
df.loc[df['CharacteristicName']=='Organic carbon', cols + ['Carbon']]

In [ ]:
# Turbidity
df.loc[df['CharacteristicName']=='Turbidity', cols + ['Turbidity']]

In [ ]:
# Sediment
df.loc[df['CharacteristicName']=='Sediment', cols + ['Sediment']]

In [ ]:
# Phosphorus
df.loc[df['TDP_Phosphorus'].notna(), ['ResultMeasureValue', 'ResultMeasure/MeasureUnitCode', 'QA_flag', 'TDP_Phosphorus']]

In [ ]:
df.loc[df['TP_Phosphorus'].notna(), ['ResultMeasureValue', 'ResultMeasure/MeasureUnitCode', 'QA_flag', 'TP_Phosphorus']]

### Combining Salinity and Conductivity

Because the number of results and variability in outliers is so variable across individual estuaries this functionality is not demonstrated here. Look to the other notebook examples.

### Datetime

datetime() formats time using dataretrieval and ActivityStart

In [ ]:
# First inspect the existing unformated fields
cols = ['ActivityStartDate', 'ActivityStartTime/Time', 'ActivityStartTime/TimeZoneCode']
df[cols]

In [ ]:
# Note the input columns are dropped (rename the result to preserve these columns)
df = clean.datetime(df)
df[['StartDate', 'Activity_datetime']]

Activity_datetime combines all three time component columns into UTC. If time is missing this is NaT so a startDate column is used to preserve date only.

### Depth

In [ ]:
# Depth of sample (default units='meter')
df = clean.harmonize_depth(df)
#df.loc[df['ResultDepthHeightMeasure/MeasureValue'].dropna(), "Depth"]
df['ResultDepthHeightMeasure/MeasureValue'].dropna()

Note: Data are often lacking sample depth metadata

### Characteristic to Column (long to wide format)

In [ ]:
# Split single QA column into multiple by characteristic (rename the result to preserve these QA_flags)
df2 = wrangle.split_col(df)
df2

In [ ]:
# This expands the single col (QA_flag) out to a number of new columns based on the unique characteristicNames and speciation
print('{} new columns'.format(len(df2.columns) - len(df.columns)))

In [ ]:
# Note: there are fewer rows because NAN results are also dropped in this step
print('{} fewer rows'.format(len(df)-len(df2)))

In [ ]:
#Examine Carbon flags from earlier in notebook (note these are empty now because NAN is dropped)
cols = ['ResultMeasureValue', 'ResultMeasure/MeasureUnitCode', 'Carbon', 'QA_Carbon']
df2.loc[df2['QA_Carbon'].notna(), cols]

Next the table is divided into the columns of interest (main_df) and characteristic specific metadata (chars_df)

In [ ]:
# split table into main and characteristics tables
main_df, chars_df = wrangle.split_table(df2)

In [ ]:
# Columns still in main table
main_df.columns

In [ ]:
# look at main table results (first 5)
main_df.head()

In [ ]:
# Empty columns that could be dropped (Mostly QA columns)
cols = list(main_df.columns)
x = main_df.dropna(axis=1, how='all')
[col for col in cols if col not in x.columns]

In [ ]:
# Join to stations to quickly aggegate/map
merge_cols = ['MonitoringLocationIdentifier', 'OrganizationIdentifier']
gdf_cols = ['geometry', 'QA_flag']
results_df = wrangle.merge_tables(main_df, stations_clipped, gdf_cols, merge_cols)

In [ ]:
# Map average temperature
results_gdf = geopandas.GeoDataFrame(results_df, geometry='geometry')
results_gdf.plot(column='Temperature', cmap='OrRd')